In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Load the cleaned data
df = pd.read_csv('../data/processed/cleaned_data.csv')

# Prepare the data (as we did in Version 2)
df.drop('customerID', axis=1, inplace=True)
X = df.drop('Churn', axis=1)
y = df['Churn']
categorical_cols = X.select_dtypes(include=['object']).columns
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Separate features (X) and target (y) for modeling
X_encoded.drop('Churn_numeric', axis=1, inplace=True)
y_encoded = df['Churn_numeric']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Define the model
rf = RandomForestClassifier()

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [10, 20, None],     # Max depth of each tree
    'min_samples_split': [2, 5, 10]  # Minimum number of samples required to split a node
}

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters found by Grid Search:")
print(best_params)
print(f"Best cross-validation accuracy: {best_score:.2f}")

# Train the final model with the best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)